https://curl.trillworks.com/

In [44]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
cookies = {
    'ASPSESSIONIDCSCSTQDS': 'FJPPKCJCCLPBLHHEEPCKPHJL',
    'rxVisitor': '1567434794791V6MI0JKUSQLU8FBP8JSQ993PDH20849S',
    '_ga': 'GA1.4.1917327044.1567434796',
    '_gid': 'GA1.4.1820181182.1567434796',
    'TS01ccf8f5': '016e3b076fe8ae3713d848c02530ec23eda76a0cb786e5843de3b2c361a764ad6066b29aefe27b13aec0d16845c52990bc09344a5324dec968ab7034535153c30f3827d1c5',
    'ASPSESSIONIDQQDTTQAR': 'CFGLCFJCCFHDPODDDJCGOCDO',
    'TS01871345': '016e3b076fd3c661d804b074e15649329e6d37810a9f1ea98494068164bd455db8e278b100ba6e6b477ce6b4128beb38dc9fae9cf562b2b8cf4f10676e7a26d4b046c7f5ff92ac30d37846c9832df25b3be6c0655b',
    'dtSa': '-',
    '_dc_gtm_UA-43178799-13': '1',
    'dtPC': '1$238641608_310h-vKHHGMLIDPLJGJIMOOEFOOLIANABGKLFO',
    'rxvt': '1567440447881|1567438520340',
    'dtLatC': '3',
    'dtCookie': '1$C2118806ECB5F998A066C983F703DD1E|5286dfffe4e737f8|1',
}

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Mobile Safari/537.36',
    'Origin': 'http://www2.bmf.com.br',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Referer': 'http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data=30/08/2019&Data1=20190830&slcTaxa=PRE',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en,en-US;q=0.9,pt-BR;q=0.8,pt;q=0.7',
}

In [14]:
# data_bmf = {}
# data_bmf['Dias Corridos'] = []
# data_bmf['DI x pré'] = {'252(2)(4)':[], '360(1)':[]}

In [64]:
date_rng = pd.date_range(start='1/1/2018', end='1/31/2018')

In [60]:
date_rng

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12',
               '2018-01-13', '2018-01-14', '2018-01-15', '2018-01-16',
               '2018-01-17', '2018-01-18', '2018-01-19', '2018-01-20',
               '2018-01-21', '2018-01-22', '2018-01-23', '2018-01-24',
               '2018-01-25', '2018-01-26', '2018-01-27', '2018-01-28',
               '2018-01-29', '2018-01-30', '2018-01-31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
days = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
# days = ['28','29','30']
# months = ['08']
# years = ['2019']

data_bmf = {}

list_df = []

for year in years:
    for month in months:
        for day in days:
            data_bmf[day+'/'+month+'/'+year] = {'Dias Corridos':[], '252(2)(4)':[], '360(1)':[]}
            params = (
                ('Data', day+'/'+month+'/'+year),
                ('Data1', year+month+day),
                ('slcTaxa', 'PRE'),
            )

            data = {
              'slcTaxa': 'PRE',
              'Data1': year+month+day,
              'Data': day+'/'+month+'/'+year,
              'convertexls1': '',
              'nomexls': '',
              'lQtdTabelas': '',
              'IDIOMA': '1'
            }

            response = requests.post('http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp', headers=headers, params=params, cookies=cookies, data=data, verify=False)
            soup = BeautifulSoup(response.text, 'html.parser')
            all_td = soup.findAll('td')

            state = 0
            for td in all_td:
                if not bool(td.attrs):
                    data_bmf[day+'/'+month+'/'+year]['Dias Corridos'].append(td.text)
                else :
                    if state == 0: 
                        data_bmf[day+'/'+month+'/'+year]['252(2)(4)'].append(td.text)
                        state = 1
                    elif state == 1:
                        data_bmf[day+'/'+month+'/'+year]['360(1)'].append(td.text)
                        state = 0
            df = pd.DataFrame(
                data=data_bmf[day+'/'+month+'/'+year].values(),
                index=data_bmf[day+'/'+month+'/'+year].keys(),
                columns=[day+'/'+month+'/'+year]*len(data_bmf[day+'/'+month+'/'+year]['Dias Corridos'])).T
            list_df.append(df)

In [5]:
df = reduce(lambda df1,df2:pd.concat([df1,df2], axis=0, join='outer'), list_df)

In [22]:
df.head()

,Dias Corridos,252(2)(4),360(1)
02/01/2009,3,"13,62","13,27"
02/01/2009,13,"13,35","13,20"
02/01/2009,28,"13,34","13,63"
02/01/2009,31,"13,34","12,88"
02/01/2009,45,"13,29","13,06"


In [8]:
df.to_csv('bmf_dataframe.csv')

In [23]:
#to_replace=r'^ba.$', value='new', regex=True
df.replace(to_replace=r'(,)', value='.', regex=True, inplace=True)

In [26]:
df.iloc[0][0]

'3'

In [38]:
df['Dias Corridos'] = df['Dias Corridos'].astype(float)
df['252(2)(4)'] = df['252(2)(4)'].astype(float)
df['360(1)'] = df['360(1)'].astype(float)

In [87]:
# day+'/'+month+'/'+year
df.reset_index().replace(to_replace=r'(^\d\d\/)', value='', regex=True).head()

,index,Dias Corridos,252(2)(4),360(1)
0,01/2009,3.0,13.62,13.27
1,01/2009,13.0,13.35,13.20
2,01/2009,28.0,13.34,13.63
3,01/2009,31.0,13.34,12.88
4,01/2009,45.0,13.29,13.06


In [88]:
import re
re.sub(r"(\d.*?)\s(\d.*?)", r"\1 \2", string1)

In [91]:
df.index = df.index.map(lambda idx: re.sub(r'(^\d\d\/)', '',idx))

In [92]:
df.head()

,Dias Corridos,252(2)(4),360(1)
01/2009,3.0,13.62,13.27
01/2009,13.0,13.35,13.20
01/2009,28.0,13.34,13.63
01/2009,31.0,13.34,12.88
01/2009,45.0,13.29,13.06
